In [ ]:
import math
import os
import machine_learning_datasets as mldatasets
import pandas as pd
import numpy as np
# Understanding The Effect of Irrelevant Features
import timeit
from tqdm.notebook import tqdm
# Filter-Based Methods
from sklearn.feature_selection import VarianceThreshold, mutual_info_classif, SelectKBest
# Embedded Methods
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression, LassoCV, LassoLarsCV, LassoLarsIC
# Wrapper Methods
from mlxtend.feature_selection import SequentialFeatureSelector
# Hybrid Methods
from sklearn.feature_selection import RFECV
# Advanced Methods
from sklearn.decomposition import PCA
import shap
from genetic_selection import GeneticSelectionCV
from scipy.stats import rankdata
# Models
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
import xgboost as xgb
# Visualize
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
X_train, X_test, y_train, y_test = mldatasets.load("nonprofit-mailer", prepare=True)
y_train = y_train.squeeze()
y_test = y_test.squeeze()

In [ ]:
rand = 9
os.environ['PYTHONHASHSEED']=str(rand)
np.random.seed(rand)
orig_plt_params = plt.rcParams
sns.set()

In [ ]:
stime = timeit.default_timer()
reg_mdl = xgb.XGBRFRegressor(max_depth=4, n_estimators=200, seed=rand)
fitted_mdl = reg_mdl.fit(X_train, y_train)
etime = timeit.default_timer()
baseline_time = etime-stime

In [ ]:
threshs = np.hstack([np.linspace(0.40,1,61), np.linspace(1.1,3,20),\
                     np.linspace(4,25,22)])

In [ ]:
y_formatter = plt.FuncFormatter(lambda x, loc: "${:,}K".format(x/1000))

profits_test = mldatasets.profits_by_thresh(y_test, reg_mdls['rf_4_all']['preds_test'],\
                                    threshs, var_costs=var_cost, min_profit=test_min_profit)
profits_train = mldatasets.profits_by_thresh(y_train, reg_mdls['rf_4_all']['preds_train'],\
                                     threshs, var_costs=var_cost, min_profit=train_min_profit)
reg_mdls['rf_4_all']['max_profit_train'] = profits_train.profit.max()
reg_mdls['rf_4_all']['max_profit_test'] = profits_test.profit.max()
reg_mdls['rf_4_all']['max_roi'] = profits_test.roi.max()
reg_mdls['rf_4_all']['min_costs'] = profits_test.costs.min()
reg_mdls['rf_4_all']['profits_train'] = profits_train
reg_mdls['rf_4_all']['profits_test'] = profits_test

mldatasets.compare_df_plots(profits_test[['costs', 'profit', 'roi']],\
                            profits_train[['costs', 'profit', 'roi']],\
                            'Test', 'Train', x_label='Threshold',\
                            y_formatter=y_formatter, plot_args={'secondary_y':'roi'})

In [ ]:
for depth in tqdm(range(5, 13)):
    mdlname = 'rf_'+str(depth)+'_all'
    stime = timeit.default_timer()
    reg_mdl = xgb.XGBRFRegressor(max_depth=depth, n_estimators=200, seed=rand)
    fitted_mdl = reg_mdl.fit(X_train, y_train)
    etime = timeit.default_timer()
    reg_mdls[mdlname] = mldatasets.evaluate_reg_mdl(fitted_mdl, X_train, X_test, y_train, y_test,\
                                            plot_regplot=False, show_summary=False, ret_eval_dict=True)
    reg_mdls[mdlname]['speed'] = (etime-stime)/baseline_time
    reg_mdls[mdlname]['depth'] = depth
    reg_mdls[mdlname]['fs'] = 'all'
    profits_test = mldatasets.profits_by_thresh(y_test, reg_mdls[mdlname]['preds_test'],\
                                    threshs, var_costs=var_cost, min_profit=test_min_profit)
    profits_train = mldatasets.profits_by_thresh(y_train, reg_mdls[mdlname]['preds_train'],\
                                    threshs, var_costs=var_cost, min_profit=train_min_profit)
    reg_mdls[mdlname]['max_profit_train'] = profits_train.profit.max()
    reg_mdls[mdlname]['max_profit_test'] = profits_test.profit.max()
    reg_mdls[mdlname]['max_roi'] = profits_test.roi.max()
    reg_mdls[mdlname]['min_costs'] = profits_test.costs.min()
    reg_mdls[mdlname]['profits_train'] = profits_train
    reg_mdls[mdlname]['profits_test'] = profits_test
    reg_mdls[mdlname]['total_feat'] = reg_mdls[mdlname]['fitted'].feature_importances_.shape[0]
    reg_mdls[mdlname]['num_feat'] = sum(reg_mdls[mdlname]['fitted'].feature_importances_ > 0)

In [ ]:
mldatasets.compare_df_plots(profits_test[['costs', 'profit', 'roi']],\
                            profits_train[['costs', 'profit', 'roi']],\
                            'Test', 'Train', x_label='Threshold',\
                            y_formatter=y_formatter, plot_args={'secondary_y':'roi'})

In [ ]:
reg_metrics_df = pd.DataFrame.from_dict(reg_mdls, 'index')\
                    [['depth', 'fs', 'rmse_train', 'rmse_test', 'max_profit_train',\
                      'max_profit_test', 'max_roi', 'min_costs', 'speed', 'num_feat']]
with pd.option_context('display.precision', 2):
    html = reg_metrics_df.sort_values(by='depth', ascending=False).style.\
        background_gradient(cmap='plasma', low=0.3, high=1, subset=['rmse_train', 'rmse_test']).\
        background_gradient(cmap='viridis', low=1, high=0.3, subset=['max_profit_train', 'max_profit_test'])
html

#### Constant Features with Variance Threshold

In [ ]:
num_cols_l = X_train.select_dtypes([np.number]).columns
cat_cols_l = X_train.select_dtypes([np.bool, np.object]).columns

num_const = VarianceThreshold(threshold=0)
num_const.fit(X_train[num_cols_l])

num_const_cols = list(set(X_train[num_cols_l].columns) - set(num_cols_l[num_const.get_support()]))

In [ ]:
cat_const_cols = X_train[cat_cols_l].nunique()[lambda x: x<2].index.tolist()
all_const_cols = num_const_cols + cat_const_cols
print(all_const_cols)

#### Quasi-Constant Features with Value-Counts

In [ ]:
thresh = 0.999
quasi_const_cols = []
num_rows = X_train.shape[0]
for col in tqdm(X_train.columns):
    top_val = (X_train[col].value_counts() /\
               num_rows).sort_values(ascending=False).values[0]
    
    if top_val >= thresh:
        quasi_const_cols.append(col)
print(quasi_const_cols)

In [ ]:
X_train_orig = X_train.copy()
X_test_orig = X_test.copy()
drop_cols = quasi_const_cols + dup_cols
X_train.drop(labels=drop_cols, axis=1, inplace=True)
X_test.drop(labels=drop_cols, axis=1, inplace=True)

### Correlation Filter-Based Methods

In [ ]:
corrs = X_train.corr(method='spearman')
print(corrs.shape)

In [ ]:
extcorr_cols = (abs(corrs) > 0.99).sum(axis=1)[lambda x: x>1].index.tolist()
print(extcorr_cols)
uncorr_cols = (abs(corrs) > 0.15).sum(axis=1)[lambda x: x==1].index.tolist()
print(uncorr_cols)

In [ ]:
corr_cols = X_train.columns[~X_train.columns.isin(uncorr_cols)].tolist()
print(len(corr_cols))

In [ ]:
mdlname = 'rf_11_f-corr'
stime = timeit.default_timer()
reg_mdl = xgb.XGBRFRegressor(max_depth=11, n_estimators=200, seed=rand)
fitted_mdl = reg_mdl.fit(X_train[corr_cols], y_train)
etime = timeit.default_timer()
reg_mdls[mdlname] = mldatasets.evaluate_reg_mdl(fitted_mdl, X_train[corr_cols], X_test[corr_cols], y_train, y_test,\
                                        plot_regplot=False, show_summary=False, ret_eval_dict=True)
reg_mdls[mdlname]['speed'] = (etime-stime)/baseline_time
reg_mdls[mdlname]['depth'] = 11
reg_mdls[mdlname]['fs'] = 'f-corr'
profits_test = mldatasets.profits_by_thresh(y_test, reg_mdls[mdlname]['preds_test'],\
                                threshs, var_costs=var_cost, min_profit=test_min_profit)
profits_train = mldatasets.profits_by_thresh(y_train, reg_mdls[mdlname]['preds_train'],\
                                threshs, var_costs=var_cost, min_profit=train_min_profit)
reg_mdls[mdlname]['max_profit_train'] = profits_train.profit.max()
reg_mdls[mdlname]['max_profit_test'] = profits_test.profit.max()
reg_mdls[mdlname]['max_roi'] = profits_test.roi.max()
reg_mdls[mdlname]['min_costs'] = profits_test.costs.min()
reg_mdls[mdlname]['profits_train'] = profits_train
reg_mdls[mdlname]['profits_test'] = profits_test
reg_mdls[mdlname]['total_feat'] = reg_mdls[mdlname]['fitted'].feature_importances_.shape[0]
reg_mdls[mdlname]['num_feat'] = sum(reg_mdls[mdlname]['fitted'].feature_importances_ > 0)

### Ranking Filter-Based Methods

In [ ]:
y_train_class = np.where(y_train > 0.68, 1, 0)

In [ ]:
mic_selection = SelectKBest(mutual_info_classif, k=160).fit(X_train, y_train_class)
mic_cols = X_train.columns[mic_selection.get_support()].tolist()
print(len(mic_cols))

In [ ]:
mdlname = 'rf_5_f-mic'
stime = timeit.default_timer()
reg_mdl = xgb.XGBRFRegressor(max_depth=5, n_estimators=200, seed=rand)
fitted_mdl = reg_mdl.fit(X_train[mic_cols], y_train)
etime = timeit.default_timer()
reg_mdls[mdlname] = mldatasets.evaluate_reg_mdl(fitted_mdl, X_train[mic_cols], X_test[mic_cols], y_train, y_test,\
                                        plot_regplot=False, show_summary=False, ret_eval_dict=True)
reg_mdls[mdlname]['speed'] = (etime-stime)/baseline_time
reg_mdls[mdlname]['depth'] = 5
reg_mdls[mdlname]['fs'] = 'f-mic'
profits_test = mldatasets.profits_by_thresh(y_test, reg_mdls[mdlname]['preds_test'],\
                                threshs, var_costs=var_cost, min_profit=test_min_profit)
profits_train = mldatasets.profits_by_thresh(y_train, reg_mdls[mdlname]['preds_train'],\
                                threshs, var_costs=var_cost, min_profit=train_min_profit)
reg_mdls[mdlname]['max_profit_train'] = profits_train.profit.max()
reg_mdls[mdlname]['max_profit_test'] = profits_test.profit.max()
reg_mdls[mdlname]['max_roi'] = profits_test.roi.max()
reg_mdls[mdlname]['min_costs'] = profits_test.costs.min()
reg_mdls[mdlname]['profits_train'] = profits_train
reg_mdls[mdlname]['profits_test'] = profits_test
reg_mdls[mdlname]['total_feat'] = reg_mdls[mdlname]['fitted'].feature_importances_.shape[0]
reg_mdls[mdlname]['num_feat'] = sum(reg_mdls[mdlname]['fitted'].feature_importances_ > 0)

In [ ]:
mldatasets.compare_df_plots(profits_test[['costs', 'profit', 'roi']],\
                            profits_train[['costs', 'profit', 'roi']],\
                            'Test', 'Train', x_label='Threshold',\
                            y_formatter=y_formatter, plot_args={'secondary_y':'roi'})

### Comparing Filter-based Methods

In [ ]:
reg_metrics_df = pd.DataFrame.from_dict(reg_mdls, 'index')\
                    [['depth', 'fs', 'rmse_train', 'rmse_test', 'max_profit_train',\
                      'max_profit_test', 'max_roi', 'min_costs', 'speed', 'total_feat', 'num_feat']]
with pd.option_context('display.precision', 2):
    html = reg_metrics_df.sort_values(by='max_profit_test', ascending=False).style.\
        background_gradient(cmap='plasma', low=0.3, high=1, subset=['rmse_train', 'rmse_test']).\
        background_gradient(cmap='viridis', low=1, high=0.3, subset=['max_profit_train', 'max_profit_test'])
html

## Exploring Embedded Feature Selection Methods

### Lasso

In [ ]:
lasso_selection = SelectFromModel(LassoCV(n_jobs=-1, random_state=rand))
lasso_selection.fit(X_train, y_train)
lasso_cols = X_train.columns[lasso_selection.get_support()].tolist()
print(len(lasso_cols))
print(lasso_cols)

### Lasso LARS

In [ ]:
llars_selection = SelectFromModel(LassoLarsCV(n_jobs=-1))
llars_selection.fit(X_train, y_train)
llars_cols = X_train.columns[llars_selection.get_support()].tolist()
print(len(llars_cols))
print(llars_cols)

### Lasso LARS with AIC

In [ ]:
llarsic_selection = SelectFromModel(LassoLarsIC(criterion='aic'))
llarsic_selection.fit(X_train, y_train)
llarsic_cols = X_train.columns[llarsic_selection.get_support()].tolist()
print(len(llarsic_cols))
print(llarsic_cols)

### Logistic Regression

In [ ]:
log_selection = SelectFromModel(LogisticRegression(C=0.0001, solver='sag',\
                                    penalty='l2', n_jobs=-1, random_state=rand))
log_selection.fit(X_train, y_train_class)
log_cols = X_train.columns[log_selection.get_support()].tolist()
print(len(log_cols))
print(log_cols)

### Fit and Evaluate Selected Models

In [ ]:
fsnames = ['e-lasso', 'e-llars', 'e-llarsic', 'e-logl2']
fscols = [lasso_cols, llars_cols, llarsic_cols, log_cols]

In [ ]:
for i, fsname in tqdm(enumerate(fsnames), total=len(fsnames)):
    depth = i + 3
    cols = fscols[i]
    mdlname = 'rf_'+str(depth)+'_'+fsname
    stime = timeit.default_timer()
    reg_mdl = xgb.XGBRFRegressor(max_depth=depth, n_estimators=200, seed=rand)
    fitted_mdl = reg_mdl.fit(X_train[cols], y_train)
    etime = timeit.default_timer()
    reg_mdls[mdlname] = mldatasets.evaluate_reg_mdl(fitted_mdl, X_train[cols], X_test[cols], y_train, y_test,\
                                            plot_regplot=False, show_summary=False, ret_eval_dict=True)
    reg_mdls[mdlname]['speed'] = (etime-stime)/baseline_time
    reg_mdls[mdlname]['depth'] = depth
    reg_mdls[mdlname]['fs'] = fsname
    profits_test = mldatasets.profits_by_thresh(y_test, reg_mdls[mdlname]['preds_test'],\
                                    threshs, var_costs=var_cost, min_profit=test_min_profit)
    profits_train = mldatasets.profits_by_thresh(y_train, reg_mdls[mdlname]['preds_train'],\
                                    threshs, var_costs=var_cost, min_profit=train_min_profit)
    reg_mdls[mdlname]['max_profit_train'] = profits_train.profit.max()
    reg_mdls[mdlname]['max_profit_test'] = profits_test.profit.max()
    reg_mdls[mdlname]['max_roi'] = profits_test.roi.max()
    reg_mdls[mdlname]['min_costs'] = profits_test.costs.min()
    reg_mdls[mdlname]['profits_train'] = profits_train
    reg_mdls[mdlname]['profits_test'] = profits_test
    reg_mdls[mdlname]['total_feat'] = reg_mdls[mdlname]['fitted'].feature_importances_.shape[0]
    reg_mdls[mdlname]['num_feat'] = sum(reg_mdls[mdlname]['fitted'].feature_importances_ > 0)

In [ ]:
reg_metrics_df = pd.DataFrame.from_dict(reg_mdls, 'index')\
                    [['depth', 'fs', 'rmse_train', 'rmse_test', 'max_profit_train',\
                      'max_profit_test', 'max_roi', 'min_costs', 'speed', 'total_feat', 'num_feat']]
with pd.option_context('display.precision', 2):
    html = reg_metrics_df.sort_values(by='max_profit_test', ascending=False).style.\
        background_gradient(cmap='plasma', low=0.3, high=1, subset=['rmse_train', 'rmse_test']).\
        background_gradient(cmap='viridis', low=1, high=0.3, subset=['max_profit_train', 'max_profit_test'])
html

### Wrapper Methods

##### Reduce Features

In [ ]:
top_cols = list(set(mic_cols).union(set(llarsic_cols)).union(set(log_cols)))
len(top_cols)

##### Sample Rows

In [ ]:
sample_size = 0.1
sample_train_idx = np.random.choice(X_train.shape[0],\
                                    math.ceil(X_train.shape[0]*sample_size),\
                                    replace=False)
sample_test_idx = np.random.choice(X_test.shape[0],\
                                    math.ceil(X_test.shape[0]*sample_size),\
                                    replace=False)

##### Sequential Forward Selection

##### Sequential Backward Selection

### Hybrid Methods

##### Recursive Feature Elimination (RFE)

##### Dimenesionality Reduction

##### Model Agnostoc Feature Importance

##### Genetic Algorithm

### Evaluating all Feature Selected Models

In [ ]:
fsnames = ['w-sfs-lda', 'w-sbs-et', 'h-rfe-lda','h-rfe-rf', 'a-pca', 'a-shap', 'a-ga-rf']
fscols = [sfs_lda_cols, sbs_et_cols, rfe_lda_cols, rfe_rf_cols, pca_cols, shap_cols, ga_rf_cols]
depths = [5, 6, 6, 6, 6, 6, 5]

In [ ]:
for i, fsname in tqdm(enumerate(fsnames), total=len(fsnames)):
    depth = depths[i]
    cols = fscols[i]
    mdlname = 'rf_'+str(depth)+'_'+fsname
    stime = timeit.default_timer()
    reg_mdl = xgb.XGBRFRegressor(max_depth=depth, n_estimators=200, seed=rand)
    fitted_mdl = reg_mdl.fit(X_train[cols], y_train)
    etime = timeit.default_timer()
    reg_mdls[mdlname] = mldatasets.evaluate_reg_mdl(fitted_mdl, X_train[cols],\
                                            X_test[cols], y_train, y_test, plot_regplot=False,\
                                            show_summary=False, ret_eval_dict=True)
    reg_mdls[mdlname]['speed'] = (etime-stime)/baseline_time
    reg_mdls[mdlname]['depth'] = depth
    reg_mdls[mdlname]['fs'] = fsname
    profits_test = mldatasets.profits_by_thresh(y_test, reg_mdls[mdlname]['preds_test'],\
                                    threshs, var_costs=var_cost, min_profit=test_min_profit)
    profits_train = mldatasets.profits_by_thresh(y_train, reg_mdls[mdlname]['preds_train'],\
                                    threshs, var_costs=var_cost, min_profit=train_min_profit)
    reg_mdls[mdlname]['max_profit_train'] = profits_train.profit.max()
    reg_mdls[mdlname]['max_profit_test'] = profits_test.profit.max()
    reg_mdls[mdlname]['max_roi'] = profits_test.roi.max()
    reg_mdls[mdlname]['min_costs'] = profits_test.costs.min()
    reg_mdls[mdlname]['profits_train'] = profits_train
    reg_mdls[mdlname]['profits_test'] = profits_test
    reg_mdls[mdlname]['total_feat'] = reg_mdls[mdlname]['fitted'].feature_importances_.shape[0]
    reg_mdls[mdlname]['num_feat'] = sum(reg_mdls[mdlname]['fitted'].feature_importances_ > 0)

In [ ]:
reg_metrics_df = pd.DataFrame.from_dict(reg_mdls, 'index')\
                    [['depth', 'fs', 'rmse_train', 'rmse_test', 'max_profit_train',\
                      'max_profit_test', 'max_roi', 'min_costs', 'speed', 'total_feat', 'num_feat']]
reg_metrics_df = reg_metrics_df[reg_metrics_df.depth < 7]
with pd.option_context('display.precision', 2):
    html = reg_metrics_df.sort_values(by='max_profit_test', ascending=False).style.\
        background_gradient(cmap='plasma', low=0.3, high=1, subset=['rmse_train', 'rmse_test']).\
        background_gradient(cmap='viridis', low=1, high=0.3, subset=['max_profit_train', 'max_profit_test'])
html